In [1]:
# automatically update imports
%load_ext autoreload
%autoreload 2

# For graphs
%matplotlib inline

# snippet for relative imports
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import extract_uses.main

extract_uses.main.main()

('Folder already exists. Assumption: Already processed, then ignoring...', 'grenoble/')


In [4]:
import pandas as pd

import loaders
import extract_uses.utils as utils

city_ref = 'grenoble'
activities_shp = city_ref+"/full_activities.shp"
residential_shp = city_ref+"/full_residential.shp"

bbox = utils.getBoundingBox(activities_shp)

geo_graph = loaders.load_graph(city_ref, bbox)
centrality_df = loaders.load_centrality(city_ref, geo_graph)
pois_df = loaders.load_extracted_osm_pois(activities_shp)
pois_residential_df = loaders.load_extracted_osm_pois(residential_shp)
all_pois_df = pd.concat([pois_df, pois_residential_df])
categories_kde_dict = loaders.load_grid_kde(city_ref, all_pois_df)

# kde_df = loaders.load_graph_kde(city_ref, geo_graph, pois_df)
# kde_residential_df = loaders.load_graph_kde(city_ref, geo_graph, pois_residential_df)


Querying locally for `['centrality']`
Found ['centrality'] stored locally
Querying locally for `['kde']`
Found ['kde'] stored locally
Querying locally for `['kde']`
Found ['kde'] stored locally


('Bounding box:', [45.11334367251151, 5.5764359, 45.24389538600425, 5.9025706])
Querying locally for `['nodes', 'edges']`
Found ['nodes', 'edges'] stored locally


In [6]:
import kde
import pandas as pd

all_pois_df = pd.concat([pois_df, pois_residential_df])
categories_kde = kde.get_categories_kde(all_pois_df, bbox)

In [118]:
categories_kde.keys()

[u'shop', u'residential', u'commercial/industrial', u'leisure/amenity']

In [106]:
keys = list(map(lambda cat : 'kde_' + str(cat), categories_kde.keys()))

In [108]:
dfs = list(map(pd.DataFrame, categories_kde.values()))

In [77]:
ser = pd.Series(categories_kde.values(), index=list(map(str, categories_kde.keys())))

In [116]:
with pd.HDFStore(loaders._generate_file_path(city_ref), 'a') as store:
    for key, df in zip(keys, dfs):
        store.put(key, df, format='table')

In [19]:
import pickle
# TODO: create a `loaders` method for `category_kde`
with open('categories_kde.p', 'wb') as f:
    pickle.dump(categories_kde, f)

In [49]:
import numpy as np

phi = 0
categories_list = categories_kde.items()
# let: n be the total number of categories; l be the total number of grid (lattice) points
n = len(categories_list[0][0])
l = len(categories_list[0][1])**2 # TODO: more `correct` GRIDSIZE handling
# [[phi += np.sum(vi - vj) for j, (kj, vj) in enumerate(categories_list[i:])] for i, (ki, vi) in enumerate(categories_list)]

for i, (ki, vi) in enumerate(categories_list):
    for j, (kj, vj) in enumerate(categories_list[i:]):
        phi += np.sum(vi - vj) # here i would divide it by l, but computationally i can do that at the end

phi_mix = phi / l # (l * n * (n-1) * .5)        

In [50]:
phi_mix

1.1384253034912237

In [53]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib qt
n_rows = len(categories_kde)
base_figsize = 8
xx, yy = kde._grid_from_bbox(bbox)

fig = plt.figure(figsize=(8, 8 * n_rows))
i = 1
for label, z in categories_kde.items():
    category_pois = all_pois_df[all_pois_df['key'] == label]
    ax1 = fig.add_subplot(n_rows, 2, i)
    ax2 = fig.add_subplot(n_rows, 2, i+1, projection='3d')
    ax1.scatter(category_pois['lon'], category_pois['lat'])
    ax1.set_ylabel(label, rotation=90)
    ax2.plot_surface(xx, yy, z, cmap=cm.RdYlBu)
    i += 2
    ax2.set_xlabel('lon', labelpad=-10)
    ax2.set_ylabel('lat', labelpad=-10)
    ax2.zaxis.set_rotate_label(False) # workaround
    ax2.set_zlabel(r'$f_c$', rotation=0, labelpad=-15)
    ax2.w_xaxis.set_ticklabels([])
    ax2.w_yaxis.set_ticklabels([])
    ax2.w_zaxis.set_ticklabels([])
    
plt.show()